# EMNIST CNN Model Training

This notebook trains the CNN model for EMNIST character recognition.

**Training Configuration:**
- **Batch size**: 128
- **Max epochs**: 30
- **Optimizer**: Adam (lr=0.001)
- **Loss**: Categorical crossentropy
- **Metrics**: Accuracy, Top-5 accuracy
- **Callbacks**: Early stopping, model checkpoint, learning rate reduction
- **Data augmentation**: Rotation (±15°), shifts (±10%), zoom (±10%)

**Target**: ≥85% test accuracy

## 重要提示：記憶體管理

由於 EMNIST 數據集較大（~697K 訓練樣本），完整訓練可能需要較多記憶體。

**選項 1**: 使用數據子集快速測試（推薦先測試）
**選項 2**: 使用完整數據集（需要足夠 RAM，約 8GB+）

本 notebook 提供兩種模式，可根據您的硬體選擇。

In [ ]:
# ========================================
# 記憶體管理配置
# ========================================

# 選擇訓練模式
USE_SUBSET = True  # True: 使用數據子集 (記憶體友好), False: 使用完整數據集

if USE_SUBSET:
    SUBSET_SIZE = 50000  # 使用 50K 訓練樣本（約佔 7%）
    BATCH_SIZE = 128
    EPOCHS = 20
    print("⚠ 使用數據子集模式")
    print(f"  訓練樣本: {SUBSET_SIZE:,} (約 7% 完整數據)")
    print(f"  預期準確率: ~80-83% (完整數據可達 85%+)")
else:
    SUBSET_SIZE = None
    BATCH_SIZE = 128
    EPOCHS = 30
    print("✓ 使用完整數據集模式")
    print(f"  需要記憶體: ~8GB+ RAM")
    print(f"  訓練時間: 20-40 分鐘")

print(f"\n配置:")
print(f"  Batch size: {BATCH_SIZE}")
print(f"  Epochs: {EPOCHS}")

## 1. Import Libraries and Setup

In [ ]:
import sys
sys.path.append('..')

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow import keras
import time
import json
from pathlib import Path
from datetime import datetime

# Import custom modules
from src.data.dataset import load_emnist
from src.preprocessing.preprocessing import (
    preprocess_data,
    create_train_val_split,
    create_data_augmentation_generator
)
from src.models.cnn import create_cnn_model, compile_model

# Set style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

print(f"TensorFlow version: {tf.__version__}")
print(f"Keras version: {keras.__version__}")
print(f"GPU available: {len(tf.config.list_physical_devices('GPU')) > 0}")
print("✓ Setup complete")

## 2. Load and Preprocess Data

In [ ]:
# Load EMNIST dataset
print("Loading EMNIST ByClass dataset...")
start_time = time.time()
x_train_full, y_train_full, x_test, y_test = load_emnist()
load_time = time.time() - start_time

print(f"✓ Dataset loaded in {load_time:.1f}s")
print(f"  Full training set: {x_train_full.shape[0]:,} samples")

# 使用子集或完整數據
if USE_SUBSET and SUBSET_SIZE is not None:
    # 隨機選擇子集（stratified sampling）
    from sklearn.model_selection import train_test_split as tts
    indices = np.arange(len(x_train_full))
    subset_indices, _ = tts(
        indices, 
        train_size=SUBSET_SIZE, 
        stratify=y_train_full, 
        random_state=42
    )
    x_train = x_train_full[subset_indices]
    y_train = y_train_full[subset_indices]
    print(f"  ⚠ Using subset: {x_train.shape[0]:,} samples ({x_train.shape[0]/x_train_full.shape[0]*100:.1f}%)")
    
    # 釋放記憶體
    del x_train_full, y_train_full
    import gc
    gc.collect()
else:
    x_train = x_train_full
    y_train = y_train_full
    print(f"  ✓ Using full dataset: {x_train.shape[0]:,} samples")

print(f"  Test: {x_test.shape[0]:,} samples")
print(f"  Image shape: {x_train.shape[1:]}")
print(f"  Number of classes: {len(np.unique(y_train))}")

In [ ]:
# Preprocess data (normalize, reshape, one-hot encode) - Memory efficient
print("\nPreprocessing data (in-place, memory-efficient)...")
start_time = time.time()

# Normalize in-place
print("  Normalizing...")
x_train = x_train.astype(np.float32) / 255.0
x_test = x_test.astype(np.float32) / 255.0

# Reshape in-place
print("  Reshaping...")
x_train = x_train.reshape(x_train.shape[0], 28, 28, 1)
x_test = x_test.reshape(x_test.shape[0], 28, 28, 1)

# One-hot encode
print("  One-hot encoding...")
from tensorflow.keras.utils import to_categorical
y_train = to_categorical(y_train, num_classes=62)
y_test = to_categorical(y_test, num_classes=62)

prep_time = time.time() - start_time

print(f"✓ Preprocessing complete in {prep_time:.1f}s")
print(f"  Training shape: {x_train.shape}")
print(f"  Training labels shape: {y_train.shape}")
print(f"  Value range: [{x_train.min():.2f}, {x_train.max():.2f}]")

In [ ]:
# Create train/validation split (85/15)
print("\nCreating train/validation split...")
x_train, x_val, y_train, y_val = create_train_val_split(
    x_train, y_train, val_size=0.15, random_state=42, stratify=True
)

print(f"✓ Split complete")
print(f"  Training: {x_train.shape[0]:,} samples (85%)")
print(f"  Validation: {x_val.shape[0]:,} samples (15%)")
print(f"  Test: {x_test.shape[0]:,} samples")

## 3. Create CNN Model

In [ ]:
# Create and compile the CNN model
print("Creating CNN model...")
model = create_cnn_model()
model = compile_model(model, learning_rate=0.001)

print("\n" + "="*70)
model.summary()
print("="*70)

# Count parameters
trainable = sum([tf.size(w).numpy() for w in model.trainable_weights])
non_trainable = sum([tf.size(w).numpy() for w in model.non_trainable_weights])
print(f"\n✓ Model created")
print(f"  Total parameters: {trainable + non_trainable:,}")
print(f"  Trainable: {trainable:,}")
print(f"  Non-trainable: {non_trainable:,}")

## 4. Setup Data Augmentation and Callbacks

In [ ]:
# Setup data augmentation
datagen = create_data_augmentation_generator(
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.1
)
datagen.fit(x_train)

print("✓ Data augmentation configured")
print("  Rotation: ±15°")
print("  Shifts: ±10%")
print("  Zoom: ±10%")

# Setup callbacks
model_path = Path('../models/emnist_cnn_v1.keras')
model_path.parent.mkdir(parents=True, exist_ok=True)

callbacks = [
    keras.callbacks.ModelCheckpoint(
        filepath=str(model_path),
        monitor='val_accuracy',
        mode='max',
        save_best_only=True,
        verbose=1
    ),
    keras.callbacks.EarlyStopping(
        monitor='val_loss',
        patience=5,
        restore_best_weights=True,
        verbose=1
    ),
    keras.callbacks.ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=3,
        min_lr=1e-6,
        verbose=1
    )
]

print("\n✓ Callbacks configured")
print("  Model checkpoint: Save best model by val_accuracy")
print("  Early stopping: Patience 5 epochs")
print("  Reduce LR: Factor 0.5, patience 3 epochs")

## 5. Train the Model

**Note**: This will take approximately 15-30 minutes depending on hardware.
Training ~594K images for up to 30 epochs with data augmentation.

In [ ]:
# Training configuration (使用上面定義的 BATCH_SIZE 和 EPOCHS)

print("="*70)
print("STARTING TRAINING")
print("="*70)
print(f"Configuration:")
print(f"  Batch size: {BATCH_SIZE}")
print(f"  Max epochs: {EPOCHS}")
print(f"  Training samples: {x_train.shape[0]:,}")
print(f"  Steps per epoch: {len(x_train) // BATCH_SIZE}")
print("="*70)

# Train the model
training_start = time.time()

history = model.fit(
    datagen.flow(x_train, y_train, batch_size=BATCH_SIZE),
    steps_per_epoch=len(x_train) // BATCH_SIZE,
    validation_data=(x_val, y_val),
    epochs=EPOCHS,
    callbacks=callbacks,
    verbose=1
)

training_time = time.time() - training_start

print("\n" + "="*70)
print(f"✓ Training completed in {training_time/60:.1f} minutes")
print("="*70)

## 6. Evaluate on Test Set

In [ ]:
# Evaluate on test set
print("Evaluating model on test set...")
test_results = model.evaluate(x_test, y_test, verbose=0)

test_loss = test_results[0]
test_accuracy = test_results[1]
test_top5_accuracy = test_results[2]

print("\n" + "="*70)
print("TEST SET RESULTS")
print("="*70)
print(f"Test loss: {test_loss:.4f}")
print(f"Test accuracy: {test_accuracy*100:.2f}%")
print(f"Test top-5 accuracy: {test_top5_accuracy*100:.2f}%")
print("="*70)

target_met = test_accuracy >= 0.85
print(f"\nTarget (≥85% accuracy): {'✓ MET' if target_met else '✗ NOT MET'}")

if target_met:
    print("🎉 Model achieves the target accuracy!")
else:
    print(f"⚠ Model needs {(0.85 - test_accuracy)*100:.2f}% more accuracy to meet target")

## 7. Visualize Training History

In [ ]:
# Plot training history
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# Accuracy
axes[0, 0].plot(history.history['accuracy'], label='Training Accuracy', linewidth=2)
axes[0, 0].plot(history.history['val_accuracy'], label='Validation Accuracy', linewidth=2)
axes[0, 0].set_xlabel('Epoch', fontsize=12)
axes[0, 0].set_ylabel('Accuracy', fontsize=12)
axes[0, 0].set_title('Model Accuracy', fontsize=14, fontweight='bold')
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3)

# Loss
axes[0, 1].plot(history.history['loss'], label='Training Loss', linewidth=2)
axes[0, 1].plot(history.history['val_loss'], label='Validation Loss', linewidth=2)
axes[0, 1].set_xlabel('Epoch', fontsize=12)
axes[0, 1].set_ylabel('Loss', fontsize=12)
axes[0, 1].set_title('Model Loss', fontsize=14, fontweight='bold')
axes[0, 1].legend()
axes[0, 1].grid(True, alpha=0.3)

# Top-5 Accuracy
axes[1, 0].plot(history.history['top_5_accuracy'], label='Training Top-5', linewidth=2)
axes[1, 0].plot(history.history['val_top_5_accuracy'], label='Validation Top-5', linewidth=2)
axes[1, 0].set_xlabel('Epoch', fontsize=12)
axes[1, 0].set_ylabel('Top-5 Accuracy', fontsize=12)
axes[1, 0].set_title('Top-5 Accuracy', fontsize=14, fontweight='bold')
axes[1, 0].legend()
axes[1, 0].grid(True, alpha=0.3)

# Learning rate (if available)
if 'lr' in history.history:
    axes[1, 1].plot(history.history['lr'], linewidth=2, color='green')
    axes[1, 1].set_xlabel('Epoch', fontsize=12)
    axes[1, 1].set_ylabel('Learning Rate', fontsize=12)
    axes[1, 1].set_title('Learning Rate Schedule', fontsize=14, fontweight='bold')
    axes[1, 1].set_yscale('log')
    axes[1, 1].grid(True, alpha=0.3)
else:
    axes[1, 1].text(0.5, 0.5, 'Learning rate not tracked', 
                     ha='center', va='center', fontsize=14)
    axes[1, 1].axis('off')

plt.tight_layout()
plt.show()

print(f"\nTraining Summary:")
print(f"  Epochs trained: {len(history.history['loss'])}")
print(f"  Best validation accuracy: {max(history.history['val_accuracy'])*100:.2f}%")
print(f"  Final validation accuracy: {history.history['val_accuracy'][-1]*100:.2f}%")
print(f"  Final training accuracy: {history.history['accuracy'][-1]*100:.2f}%")

## 8. Save Training History

In [ ]:
# Save training history to JSON
history_path = Path('../models/emnist_cnn_v1_history.json')

history_dict = {
    'history': {k: [float(v) for v in vals] for k, vals in history.history.items()},
    'config': {
        'batch_size': BATCH_SIZE,
        'epochs': len(history.history['loss']),
        'learning_rate': 0.001,
        'val_size': 0.15,
        'use_augmentation': True,
        'use_subset': USE_SUBSET,
        'subset_size': SUBSET_SIZE if USE_SUBSET else None,
        'training_time_seconds': training_time
    },
    'test_results': {
        'test_loss': float(test_loss),
        'test_accuracy': float(test_accuracy),
        'test_top5_accuracy': float(test_top5_accuracy)
    },
    'timestamp': datetime.now().isoformat()
}

with open(history_path, 'w') as f:
    json.dump(history_dict, f, indent=2)

print(f"✓ Training history saved to {history_path}")
print(f"✓ Best model saved to {model_path}")

if USE_SUBSET:
    print(f"\n⚠ Note: Trained with {SUBSET_SIZE:,} samples subset")
    print(f"  For ≥85% accuracy, retrain with USE_SUBSET=False")

print("\n" + "="*70)
print("TRAINING COMPLETE!")
print("="*70)